# Model defining and fitting

In [ ]:
import torch
from torch import nn, optim
import numpy as np
from scipy import stats as ss
from matplotlib import pyplot as plt
from IPython.display import clear_output

In [ ]:
from random import random
class Generator:
  def __init__(self, n_bidders, batch_size, dist, params={}):
    self.n_bidders = n_bidders
    self.batch_size = batch_size
    self.dist = dist
    self.params = params 

  def generate(self, seed=np.random.randint(1, 15000)):
    if self.dist == 'irregular':
      probs, params, dist_name = self.params['probs'], self.params['params'], self.params['dist_name']
      gen = lambda x: dist_name.rvs(**x)
      gen = np.vectorize(gen)
      dist_matrix = np.random.choice(params, size=(self.batch_size, self.n_bidders), p=probs)
      sample_val = np.array(list(map(gen, dist_matrix)))
      return torch.tensor(sample_val, dtype=torch.float)

    return torch.tensor(
        self.dist.rvs(size=(self.batch_size, self.n_bidders), random_state=seed, **self.params), dtype=torch.float
    )

In [ ]:
class MyersonNetV4(nn.Module):
    def __init__(self, n_bidders, J_functions=10, K_groups=10, B=1.0, softmax_temperature=1.0):
      super(MyersonNetV4, self).__init__()
      self.J, self.K, self.B, self.kappa, self.n_bidders = J_functions, K_groups, B, softmax_temperature, n_bidders
      self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
      self.w_transformation = nn.Sequential(
          nn.Linear(self.n_bidders, self.J * self.K),
          nn.ReLU(),
          nn.Linear(self.J * self.K, self.J * self.K),
          nn.ReLU(),
          nn.Linear(self.J * self.K, self.J * self.K),
      )
      self.b_transformation = nn.Sequential(
          nn.Linear(self.n_bidders, self.J * self.K),
          nn.ReLU(),
          nn.Linear(self.J * self.K, self.J * self.K),
          nn.ReLU(),
          nn.Linear(self.J * self.K, self.J * self.K),
      )

      self.mask = torch.ones([self.n_bidders, 1, self.n_bidders]).to(self.device)
      self.mask[np.arange(self.n_bidders), :, np.arange(self.n_bidders)] = 0.

      
    def transform_weights(self, data):
      w = self.w_transformation(data).view(self.J, self.K, -1, 1)
      bias = self.b_transformation(data).view(self.J, self.K, -1, 1)
      return (w, bias)
    
    def get_virtual_valuation(self, data, weights):
      weights, bias = weights
      return torch.min(
          torch.max(torch.exp(weights) * data + bias, axis=0).values,
          axis=0
      ).values

    def get_inverse_virtual_valuation(self, phi, weights):
      weights, bias = weights
      return torch.max(
          torch.min(torch.exp(-weights) * (phi - bias), axis=0).values,
          axis=0
      ).values

    def get_allocation(self, array, task):
        mask = torch.ones(array.shape[1]+1).to(self.device)
        mask[-1] = 0.
        mask = torch.diag(mask)[:-1, :]
        if task == 'train':
          return torch.nn.functional.softmax(array @ (self.kappa * mask), dim=-1)[:, :-1]
        elif task == 'eval':
          return torch.nn.functional.softmax(array @ (1e25 * mask), dim=-1)[:, :-1]
        else:
          raise ValueError('Wrong task, choose among "train", "eval".')
        

    def get_payment(self, phi, weights):
      second_price = torch.max(self.mask * phi, axis=-1).values.swapaxes(0, 1)
      payment = self.get_inverse_virtual_valuation(second_price, weights)
      return payment

    def forward(self, data, task='train'):
      weights = self.transform_weights(data)
      phi = self.get_virtual_valuation(data, weights)
      allocation = self.get_allocation(phi, task) 
      payment = self.get_payment(phi, weights)
      return - torch.mean(
          torch.sum(allocation * payment, axis=1)
      ), (payment, allocation, data, phi)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
n_bidders, J_functions, K_groups, B, softmax_temperature, batch_size = 3, 15, 33, 8, 1e2, 128
                                                                          
model = MyersonNetV4(n_bidders, J_functions, K_groups, B, softmax_temperature).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-5, )
revenue = [0]
utilities = [0]

In [ ]:
torch.autograd.set_detect_anomaly(True)
%mkdir models_at_iter
for i in range(0, 7000):
    batch = torch.cat([generator.generate() for generator in generators])[torch.randperm(1280)].to(device)
    optimizer.zero_grad()
    neg_revenue, (payment, allocation, data, phi) = model(batch)
    neg_utility = -(allocation * (data  - payment)).sum(axis=-1).mean()
    try:
      neg_revenue.backward()
      optimizer.step()
    except:
      continue
    revenue.append(- neg_revenue.to('cpu').detach().numpy().item()/batch.mean().item())
    utilities.append(- neg_utility.to('cpu').detach().numpy().item()/batch.mean().item())
    if i%50 == 0:
      print(i, 'revenue = ', revenue[-1])
      print(i, 'utility=', utilities[-1])
      torch.save(model.state_dict(), '/content/models_at_iter/model_at_iter_{}_15_33.pickle'.format(i))

### Plot results

In [ ]:
import pandas as pd
fig = plt.figure(figsize=(15, 5))
plt.plot(np.arange(len(revenue[1:]))* 2, revenue[1:], label='revenue')
plt.plot(np.arange(len(revenue[1:]))* 2, utilitys[1:], label='utility')
plt.title('Process of convergence')
plt.xlabel('Iteration')
plt.ylabel('Welfare/Revenue')
plt.legend()

### Difficulty plot

In [ ]:
eval_dataset = [[generator.generate().to(device) for generator in generators] for i in range(100)]

In [ ]:
revenue_achieved = [[-model(item, 'eval')[0].item()/item.mean().item() for item in x ] for x in eval_dataset]

In [ ]:
curve = pd.melt(pd.DataFrame(revenue_achieved, columns = 3 + np.linspace(0, 30, 10))).rename(columns={'variable':'right_border', 'value':'complexity of an auction'})

In [ ]:
import seaborn as sns
sns.set_theme(color_codes=True)
fig, ax = plt.subplots(figsize=(20, 10))
sns.lineplot(data=curve, x='right_border', y='complexity of an auction', ax=ax)
plt.title('Complexity of an optimapl auction versus distribution range')
plt.show()